# Working with MIO Solvers

Consider the simple knapsack problem. Iain wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}

How would you solve this? The simple way is just to consider each possible value for $x$ and compare the cost. After Iain has weighed all $2^N$ possible collections of items (and verified that he can lift them at once), he just chooses the best set. If $N$ is large though, this could take a while (or even forever!)

A mixed-integer solver is able to solve this problem to provable optimality very fast (much faster than forever) without having to check all possible solutions. How? It explores possible solutions and keeps track of two things:

- The best solution found so far
- A bound on the best possible solution value

Using these two pieces of information, the solver can avoid checking certain solutions if it knows they won't help.

For example, suppose we have found a knapsack solution with value \$10. We now know that we can ignore any solution if it can't go higher than \$10. If the total value of all items is \$20 and we have excluded more than \$10 of value from the solution, we don't need to enumerate this line of solutions any further.

You can use MIP solvers without knowing how they work, but having some sense of what's going on inside the solver helps to understand your problem better, and especially figure out how "difficult" it is.

Let's solve our simple knapsack problem for Iain and see what the solver spits out.

In [ ]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@variable(m, x, Bin)
@variable(m, y, Bin)

@constraint(m, x + 2y ≤ 1.5)
@objective(m, Max, x + y)

solve(m)

What's going on here? First, Gurobi is telling us some summary statistics about our problem:
```
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range    [1e+00, 2e+00]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+00, 2e+00]
```
Next, we see that Gurobi ran some heuristic procedure _before_ it began solving, and produced a feasible solution with value 1:
```
Found heuristic solution: objective 1
```
Next, Gurobi runs presolve and removes 1 row and 2 columns: in other words, it removes everything! (It also does this really quickly).
```
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
```
Since there isn't anything left to the problem, it doesn't have to do any work to solve the problem, and it does this about as fast as you would expect:
```
Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)
```
Gurobi is done solving the problem, so now it tells us some summary statistics: the number of solutions, the objective value of the best feasible solution, the best upper bound, and the percent gap between the two:
```
Solution count 1: 1 
Pool objective bound 1

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0%
```

This is kind of dull since Gurobi solves this with a simple heuristic! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=350$, naive enumeration would create $2^{350}\sim 10^{105}$ nodes, which would take quite some time (there are about $10^{80}$ atoms in the universe!). How does the solver actually tackle it?

In [ ]:
srand(100)
N = 350

m = Model(solver=GurobiSolver())
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)

The stuff at the top is mostly the same, but now we force the solver to actually do some work. First, it finds an alright heuristic solution:
```
Found heuristic solution: objective 6.86518
```
Presolve isn't able to do much of anything (probably because the problem is dense):
```
Presolve time: 0.00s
Presolved: 10 rows, 350 columns, 3500 nonzeros
Variable types: 0 continuous, 350 integer (350 binary)
```
Then it solves the relaxation and reports back:
```
Root relaxation: objective 1.599653e+01, 33 iterations, 0.00 seconds
```
Now it explores the branch-and-bound tree, and updates us as it goes along:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
H    0     0                      15.0893488   15.99653  6.01%     -    0s
H    0     0                      15.4881914   15.99653  3.28%     -    0s
     0     0   15.99041    0   13   15.48819   15.99041  3.24%     -    0s
     0     0   15.99041    0   10   15.48819   15.99041  3.24%     -    0s
     0     0   15.97624    0   13   15.48819   15.97624  3.15%     -    0s
     0     0   15.97624    0   14   15.48819   15.97624  3.15%     -    0s
     0     0   15.97624    0   14   15.48819   15.97624  3.15%     -    0s
     0     2   15.97624    0   14   15.48819   15.97624  3.15%     -    0s
H55075 18672                      15.4995833   15.76729  1.73%   3.9    2s
H105392 25979                      15.5156084   15.70839  1.24%   4.0    4s
 152394 29371   15.62812   38   10   15.51561   15.66818  0.98%   4.0    5s
```
What does this all mean? Let's look at just the first line:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
```
We see that the information is broken down into four main columns:

1. ``Nodes``: Global node information
    * how many nodes have we looked at
    * how many do we have in our queue
2. ``Current Node``
    * objective
    * depth in the tree
    * number of noninteger variables in the solution
3. ``Objective Bounds``
    * Best incumbent (lower bound)
    * node upper bound
    * the gap between the two
4. ``Work``
    * average simplex iterations per node
    * total elapsed time
    
Most of the time, we are interested in how the `Incumbent` and `BestBd` columns are changing. The `Gap` column gives the percentage difference between these values, and we will be finished solving when the gap reaches zero.

Finally, we get a neat summary of the cutting planes Gurobi found useful:
```
Cutting planes:
  Gomory: 12
```
This cutting plane information isn't really useful unless you have a very strong understanding of how the MIP solver works.

```
Explored 263070 nodes (1032545 simplex iterations) in 7.44 seconds
Thread count was 8 (of 8 available processors)
```
All told, we explored 263,070  nodes, much less than the $2^{350}$ we were worried about. All this only took 103,2545 simplex iterations and 7.44 seconds, which shows the power of the solver.

```
Solution count 5: 15.5156 15.4996 15.4882 ... 6.86518
Pool objective bound 15.5156
```
Gurobi found five feasible solutions while it was exploring. Note that these are **not** the five best solutions to the problem. We don't explore most of the possible solutions, so we won't neccessarily run into the second or third best solutions on our way to the optimal solution. There is a feature in Gurobi to generate the $k$ best solutions to a problem, but this is an advanced feature we might cover in the future.

Now what about those ``H``s that appear? That tells us that Gurobi ran a heuristic and found a new best solution. You can see for yourself, as the incumbent value increases while the bound remains the same; we also don't get any ``Current Node`` information:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
H    0     0                      15.0893488   15.99653  6.01%     -    0s
```
You'll also sometimes see a ``*`` instead of the ``H``, which says that the feasible solution came from branching instead of heuristics.

Gurobi likes to spare your screen, so it doesn't dump information about every node, but will update you periodically as it works through the tree.

# Solver parameters: Should you bother?

Gurobi (and other high-quality solvers) allow you to tweak a wide range of different parameters; _sometimes_ tuning these can drastically improve performance. It can be kind of intimidating, though: Gurobi has over 100 parameters, so which are the important ones?

Some useful ones:

* ``OutputFlag``: set to `1` to hide all solver output
* ``TimeLimit``: how long the solver will run before giving up
* ``MIPGap``: stop solving when the objective is within this fraction of optimality (e.g. 0.05 means stop when within 5% of the optimal solution, the default is 0.0001 so it stops when within 0.01% of optimality)
* ``MIPFocus``: High-level controls on solver priority (proving optimality or increasing bound or finding optimal solution)

Is that it? Well, no, but you probably need domain knowledge about your problem to go much further. There's an alternative: Gurobi has a parameter tuning feature you can try to "learn" good parameter settings for a particular model. Try it out if you aren't quite happy with your performance.

How do you set parameters?

In [ ]:
srand(100)
N = 350

m = Model(solver=GurobiSolver(TimeLimit=1))
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)